# Import necessaty Liberaries

In [1]:
%%bash
# pip install numpy==1.19.5

pip3 -qqq install transformers[torch]

pip3 -qqq install gdcm
pip3 -qqq install pydicom
pip -qqq install faiss-gpu

pip install -q tensorflow-recommenders
pip install -q --upgrade tensorflow-datasets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import tensorflow as tf
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import faiss
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds
from typing import Dict, Text
import tensorflow_hub as hub

In [3]:
print(np.__version__)

1.26.4


In [4]:
# Set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Import and Explore Dataset

## Import Datasets

In [5]:
users = pd.read_csv('/content/users_small.csv')
videos = pd.read_csv('/content/videos_small.csv')
users_videos = pd.read_csv('/content/user_video.csv')

In [6]:
users.head()

,User ID,Country,Age,Gender,Device
0,1,United States,28,Male,Smartphone
1,2,Canada,35,Female,Tablet
2,3,United Kingdom,42,Male,Smart TV
3,4,Australia,51,Female,Laptop
4,5,Germany,33,Male,Smartphone


In [7]:
videos.head()

,Title,video_url,Published At,Keyword,Likes,Comments,Views,is_valid,valid_video,video_duration
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,tech,3407.0,672.0,135612.0,True,True,243.0
1,The most EXPENSIVE thing I own.,b3x28s61q3c,2022-08-24,tech,76779.0,4306.0,1758063.0,True,True,143.0
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,tech,63825.0,3338.0,1564007.0,True,True,1068.0
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,2022-08-23,tech,71566.0,1426.0,922918.0,True,True,1034.0
4,Best Back to School Tech 2022!,ErMwWXQxHp0,2022-08-08,tech,96513.0,5155.0,1855644.0,True,True,1272.0


In [8]:
users_videos.head()

,user_id,video_id,watch_duration
0,39,48,659
1,29,151,402
2,15,62,345
3,8,37,146
4,21,99,863


In [9]:
users_videos.video_id.value_counts().sort_index()

,count
video_id,
1,1
2,1
3,1
5,1
12,1
13,1
15,1
24,1
27,2


## Make some changes on dataset

In [10]:
videos.drop(columns=['is_valid', 'valid_video', 'Published At'], inplace=True)
videos.head()

,Title,video_url,Keyword,Likes,Comments,Views,video_duration
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,tech,3407.0,672.0,135612.0,243.0
1,The most EXPENSIVE thing I own.,b3x28s61q3c,tech,76779.0,4306.0,1758063.0,143.0
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,tech,63825.0,3338.0,1564007.0,1068.0
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,tech,71566.0,1426.0,922918.0,1034.0
4,Best Back to School Tech 2022!,ErMwWXQxHp0,tech,96513.0,5155.0,1855644.0,1272.0


In [11]:
# Python3 Program to Create list
# with integers within given range
import itertools
r1=1
r2=175
#using the chain function from the itertools module
numbers = list(itertools.chain(range(r1, r2+1)))
print(numbers)
#This code is contributed by Edula Vinay Kumar Reddy


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175]


In [12]:
videos["video_id"] = numbers
videos.head()

,Title,video_url,Keyword,Likes,Comments,Views,video_duration,video_id
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,tech,3407.0,672.0,135612.0,243.0,1
1,The most EXPENSIVE thing I own.,b3x28s61q3c,tech,76779.0,4306.0,1758063.0,143.0,2
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,tech,63825.0,3338.0,1564007.0,1068.0,3
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,tech,71566.0,1426.0,922918.0,1034.0,4
4,Best Back to School Tech 2022!,ErMwWXQxHp0,tech,96513.0,5155.0,1855644.0,1272.0,5


In [13]:
users.rename(columns={"User ID": "user_id"}, inplace=True)

## Merge all datasets

In [14]:
users_videos.head()

,user_id,video_id,watch_duration
0,39,48,659
1,29,151,402
2,15,62,345
3,8,37,146
4,21,99,863


In [15]:
videos.head()

,Title,video_url,Keyword,Likes,Comments,Views,video_duration,video_id
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,tech,3407.0,672.0,135612.0,243.0,1
1,The most EXPENSIVE thing I own.,b3x28s61q3c,tech,76779.0,4306.0,1758063.0,143.0,2
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,tech,63825.0,3338.0,1564007.0,1068.0,3
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,tech,71566.0,1426.0,922918.0,1034.0,4
4,Best Back to School Tech 2022!,ErMwWXQxHp0,tech,96513.0,5155.0,1855644.0,1272.0,5


In [16]:
users.head()

,user_id,Country,Age,Gender,Device
0,1,United States,28,Male,Smartphone
1,2,Canada,35,Female,Tablet
2,3,United Kingdom,42,Male,Smart TV
3,4,Australia,51,Female,Laptop
4,5,Germany,33,Male,Smartphone


In [17]:
df1 = pd.merge(videos, users_videos, on='video_id')
df2 = pd.merge(df1, users, on='user_id')
df = df2
df.head()

,Title,video_url,Keyword,Likes,Comments,Views,video_duration,video_id,user_id,watch_duration,Country,Age,Gender,Device
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,tech,3407.0,672.0,135612.0,243.0,1,36,103,France,35,Male,Tablet
1,President Zelenskyy promises victory over Russ...,95MMQzBkE30,news,588.0,887.0,69951.0,1350.0,51,36,821,France,35,Male,Tablet
2,Best Of Boxing 2021 | Full Episode | SHOWTIME ...,YClWEHi6mzY,sports,6568.0,591.0,592998.0,1773.0,129,36,1445,France,35,Male,Tablet
3,The most EXPENSIVE thing I own.,b3x28s61q3c,tech,76779.0,4306.0,1758063.0,143.0,2,9,37,Spain,37,Male,Smartphone
4,INVENTIONS OF THE FUTURE THAT WILL SOON BE AVA...,_f8Gb-vojoU,tech,42075.0,1820.0,7188648.0,515.0,38,9,191,Spain,37,Male,Smartphone


# Extract features from dataset

## CBOW vectorizer

In [18]:
# CBOW function
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,\
	Embedding, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def CBOW(corpus, column_name):

  # Convert the corpus to a sequence of integers
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(corpus)
  sequences = tokenizer.texts_to_sequences(corpus)

  # Define the parameters
  vocab_size = len(tokenizer.word_index) + 1
  embedding_size = 10
  window_size = 2

  # Generate the context-target pairs
  contexts = []
  targets = []
  for sequence in sequences:
    for i in range(window_size, len(sequence) - window_size):
      context = sequence[i - window_size:i] +\
        sequence[i + 1:i + window_size + 1]
      target = sequence[i]
      contexts.append(context)
      targets.append(target)

  # Convert the contexts and targets to numpy arrays
  X = np.array(contexts)

  # Define the CBOW model
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size,
            output_dim=embedding_size,
            input_length=2*window_size))
  model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
  model.add(Dense(units=vocab_size, activation='softmax'))
  model.save_weights(f'cbow_weights_{column_name}.h5')


  # Load the pre-trained weights
  model.load_weights(f'cbow_weights_{column_name}.h5')

  # Get the word embeddings
  embeddings = model.get_weights()[0]

  # Perform PCA to reduce the dimensionality
  # of the embeddings
  pca = PCA(n_components=2)
  reduced_embeddings = pca.fit_transform(embeddings)

  return reduced_embeddings

## DistilBert Embedding

In [19]:
def get_BERT_model_info(model_ID, device):
  # Save the model to device
  model = DistilBertModel.from_pretrained(model_ID).to(device)
  # Get the tokenizer
  tokenizer = DistilBertTokenizer.from_pretrained(model_ID)

  return model, tokenizer

In [20]:
Bert_model, Bert_tokenizer = get_BERT_model_info("bert-base-multilingual-uncased", device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertModel were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'transformer.layer.0.attention.k_lin.bias', 'tr

In [21]:
def get_single_text_embedding(text):
  inputs = Bert_tokenizer(text, return_tensors='pt').to(device)
  text_embeddings = Bert_model(**inputs)
  # Convert the embeddings to numpy array
  embedding_as_np = text_embeddings[0].cpu().detach().numpy()
  embedding_as_np = np.median(embedding_as_np, axis=1)

  return embedding_as_np

In [22]:
def get_all_text_embeddings(df, text_col):
  df[text_col + '_embeddings'] = df[str(text_col)].apply(get_single_text_embedding)
  return df

## OneHot Encoder

In [23]:

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example

def onehotencoder(data):
  values = array(data)
  # print(values)
  # integer encode
  label_encoder = LabelEncoder()
  integer_encoded = label_encoder.fit_transform(values)
  # print(integer_encoded)
  # binary encode
  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
  # print(onehot_encoded)
  return onehot_encoded

In [24]:
def get_all_onehot(df, text_col):
  df[text_col + '_onehot'] = onehotencoder(df[str(text_col)]).tolist()
  return df

## Bucketize

In [25]:
def age_bucketize_and_onehot_encode(data):
  bins = pd.IntervalIndex.from_tuples([(0, 10), (10, 15), (15, 20), (20, 30), (30, 60), (60, 100)])
  bucket_ages = pd.cut(data, bins)
  return onehotencoder(pd.Series(bucket_ages))

In [26]:
def get_all_age_bucketz_encoded(df, text_col):
  df[text_col + '_encoded'] = age_bucketize_and_onehot_encode(df[str(text_col)]).tolist()
  return df

## Extract features using defined functions

In [27]:
# Extract feature from 'title' column
df = get_all_text_embeddings(df, "Title")

In [28]:
# Bucketize and encode 'Age' column
df = get_all_age_bucketz_encoded(df, "Age")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [29]:
# OneHot 'Gender' column
df = get_all_onehot(df, "Gender")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [30]:
# OneHot 'Device' column
df = get_all_onehot(df, "Device")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Modeling

## Create DataSet

### Split data into train and test

In [31]:
import numpy as np
def my_func(arg):
  arg = tf.cast(arg, dtype=tf.float32)
  arg = tf.convert_to_tensor(arg, dtype=tf.float32)
  return tf.expand_dims(arg, axis=1)

In [32]:
df_final = df.drop(columns=['Title', 'video_url', 'Keyword', 'Likes', 'Comments', 'Views', 'Age', 'Gender', 'Device'])

In [33]:
df_final.head(1)

,video_duration,video_id,user_id,watch_duration,Country,Title_embeddings,Age_encoded,Gender_onehot,Device_onehot
0,243.0,1,36,103,France,"[[-0.6272644, -0.000942694, -0.36949426, -0.42...","[0.0, 1.0]","[0.0, 1.0]","[0.0, 0.0, 0.0, 1.0]"


In [34]:
class MyDataFrameDataset(tfds.core.GeneratorBasedBuilder):
    """DatasetBuilder for a dataset created from Pandas DataFrames."""

    VERSION = tfds.core.Version('1.0.0')

    def __init__(self, df, **kwargs):
        self.df = df
        super().__init__(**kwargs)

    def _info(self):
        return tfds.core.DatasetInfo(
            builder=self,
            description=("This is a custom dataset created from Pandas DataFrames."),
            features=tfds.features.FeaturesDict({
              "user_id": tf.int64,
              "age": tfds.features.Tensor(shape=(2,), dtype=tf.float32),
              "gender": tfds.features.Tensor(shape=(2,), dtype=tf.float32),
              "device": tfds.features.Tensor(shape=(4,), dtype=tf.float32),
              "country": tfds.features.Text(),
              "video_id": tf.int64,
              "video_duration": tf.int64,
              "title": tfds.features.Tensor(shape=(768,), dtype=tf.float32),
              "watch_duration": tf.int64
            }),
            supervised_keys=None,
        )

    def _split_generators(self, dl_manager):
        return {
            'full': self._generate_examples(self.df)
        }

    def _generate_examples(self, df):
        """Yields examples."""
        for idx, row in df.iterrows():
            yield idx, {
                'user_id': row['user_id'],
                'age': row['Age_encoded'],
                'gender': row['Gender_onehot'],
                'device': row['Device_onehot'],
                'country': row['Country'],
                'video_id': row['video_id'],
                'video_duration':row['video_duration'],
                'title': list(np.squeeze(row['Title_embeddings'])),
                'watch_duration': row['watch_duration']
            }

In [51]:
builder = MyDataFrameDataset(df=df_final)
builder.download_and_prepare()
datasets_full = builder.as_dataset()

In [52]:
datasets_full

{'full': <_PrefetchDataset element_spec={'age': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'country': TensorSpec(shape=(), dtype=tf.string, name=None), 'device': TensorSpec(shape=(4,), dtype=tf.float32, name=None), 'gender': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'title': TensorSpec(shape=(768,), dtype=tf.float32, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'video_duration': TensorSpec(shape=(), dtype=tf.int64, name=None), 'video_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'watch_duration': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}

In [54]:
# Extract counties for fedd into TextVectorizer
countries = datasets_full['full'].map(lambda x: x['country'])

In [53]:
datasets = datasets_full['full'].map(lambda x: {
    'user_id': x['user_id'],
    'country': x['country'],
    'age': x['age'],
    'gender': x['gender'],
    'device': x['device'],
    'title': x['title'],
    'video_duration': x['video_duration'],
    'video_id': x['video_id'],
    'watch_duration': x['watch_duration']
})

In [56]:
tf.random.set_seed(42)

train = datasets.take(80)
test = datasets.skip(80).take(20)


cached_train = train.batch(8).cache()
cached_eval = test.batch(8).cache()

country = countries.batch(8)
user_id = datasets.batch(8).map(lambda x: x['user_id'])

video_id = datasets.batch(8).map(lambda x: x['video_id'])

unique_country = np.unique(np.concatenate(list(country)))
unique_user_id = np.unique(np.concatenate(list(user_id)))
unique_video_id = np.unique(np.concatenate(list(video_id)))

In [49]:
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
    "timestamp": x["timestamp"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/incomplete.22NFR2_0.1.1/movielens-train.tfrecord*...…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [64]:
for rat in ratings.take(1):
  print(rat['movie_title'])

tf.Tensor(b"One Flew Over the Cuckoo's Nest (1975)", shape=(), dtype=string)


In [42]:
for example in cached_train.take(1):
  user_id_ex = example['user_id']
  country_ex = example['country']
  age_ex = example['age']
  gender_ex = example['gender']
  device_ex = example['device']
  title_ex = example['title']
  video_duration_ex = example['video_duration']
  video_id_ex = example['video_id']
  watch_duration_ex = example['watch_duration']
  # print(list(example.keys()))
  # print(f"user_id: {example['user_id']}, {example['user_id'].shape}, {example['user_id'].dtype}, {tf.is_tensor(example['user_id'])}")
  # print(f"country: {example['country']}, {example['country'].shape}")
  # print(f"age: {example['age']}, {example['age'].shape}")
  # print(f"gender: {example['gender']}, {example['gender'].shape}")
  # print(f"device: {example['device']}, {example['device'].shape}")
  # print(f"title: {example['title']}, {example['title'].shape}")
  # print(f"video_duration: {example['video_duration']}, {example['video_duration'].shape}")
  # print(f"video_id: {example['video_id']}, {example['video_id'].shape}")
  # print(f"watch_duration: {example['watch_duration']}, {example['watch_duration'].shape}")

## Create Models

In [ ]:
max_tokens = 10_000


# Define the TextVectorization layer
country_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens
)

# Fit the TextVectorization layer to your data
country_vectorizer.adapt(country_ex)

# Use the fitted TextVectorization layer in the country_embedding model
country_embedding = tf.keras.Sequential([
    country_vectorizer,
    tf.keras.layers.Embedding(input_dim=max_tokens, output_dim=32, mask_zero=True),
    tf.keras.layers.GlobalAveragePooling1D(),
])

# Now you can use the country_embedding model without encountering the error
country_embedded = country_embedding(country_ex)

In [ ]:
# user_model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=max(unique_user_id) + 1, output_dim=32, input_length=1)
# ])

# user_embedding = user_model(user_id_ex)


In [ ]:
# country_model = tf.keras.Sequential([
#     tf.keras.layers.StringLookup(
#         vocabulary=unique_country, mask_token=None
#     ),
#     tf.keras.layers.Embedding(len(unique_country) + 1, 32)
# ])

# country_embedding = country_model(country_ex)

In [ ]:
# age_model = tf.keras.Sequential([
#     # tf.keras.layers.InputLayer(shape=(2,), batch_size=8)
#     tf.keras.Input(shape=(2,))
# ])

# age_output = age_model(age_ex)

In [ ]:
# device_model = tf.keras.Sequential([
#     tf.keras.Input(shape=(4,))
# ])

# device_output = device_model(np.asarray(device_ex))

In [ ]:
# gender_model = tf.keras.Sequential([
#     tf.keras.Input(shape=(2,))
# ])

# gender_output = gender_model(np.asarray(gender_ex))

In [ ]:
# max_length = 32

# age_padded = tf.pad(age_output, ([0, 0], [0, max_length - tf.shape(age_output)[1]]))
# device_padded = tf.pad(device_output, ([0, 0], [0, max_length - tf.shape(device_output)[1]]))
# gender_padded = tf.pad(gender_output, ([0, 0], [0, max_length - tf.shape(gender_output)[1]]))
# country_padded = tf.pad(country_embedding, ([0, 0], [0, max_length - tf.shape(country_embedding)[1]]))
# user_id_padded = tf.pad(tf.squeeze(user_embedding), ([0, 0], [0, max_length - tf.shape(tf.squeeze(user_embedding))[1]]))

In [ ]:
# tf.concat([my_func(user_id_padded), my_func(country_padded), my_func(age_padded), my_func(gender_padded), my_func(device_padded)], axis=1).shape

In [ ]:
# video_id_model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=max(unique_video_id) + 1, output_dim=32, input_length=1)
# ])

# video_id_embedding = video_id_model(np.asarray(video_id_ex))

In [ ]:
# title_model = tf.keras.Sequential([
#     tf.keras.Input(shape=(768,))
# ])

# title_output = title_model(np.asarray(title_ex))

In [ ]:
# video_duration_model = tf.keras.Sequential([
#     tf.keras.Input(shape=(1,))
# ])

# video_duration_output = video_duration_model(tf.reshape(tf.convert_to_tensor([tf.cast(video_duration_ex, dtype=tf.float32)], dtype=tf.float32), [8, 1]))

In [ ]:
    # # Use tf.pad instead of tf.Variable
    # max_length = 768  # Adjust this value as needed
    # video_id_padded = tf.pad(tf.squeeze(video_id_embedding), [[0, 0], [0, max_length - tf.shape(tf.squeeze(video_id_embedding))[1]]])
    # video_duration_padded = tf.pad(video_duration_output, [[0, 0], [0, max_length - tf.shape(video_duration_output)[1]]])
    # title_padded = tf.pad(title_output, [[0, 0], [0, max_length - tf.shape(title_output)[1]]])

In [ ]:
# tf.concat([my_func(video_id_padded), my_func(title_padded), my_func(video_duration_padded)], axis=1).shape

In [76]:
class UserModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    max_tokens = 10_000

    self.user = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=max(unique_user_id) + 1, output_dim=32, input_length=1)
    ])

    self.country= tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_country,mask_token=None),
      tf.keras.layers.Embedding(len(unique_country) + 1, 32)
    ])

    self.country_vectorizer = tf.keras.layers.TextVectorization(
            max_tokens=max_tokens)

    self.country_text = tf.keras.Sequential([
      self.country_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.country_vectorizer.adapt(countries)

    self.age = tf.keras.Sequential([
        tf.keras.Input(shape=(2,))
    ])
    self.device = tf.keras.Sequential([
        tf.keras.Input(shape=(4,))
    ])
    self.gender = tf.keras.Sequential([
        tf.keras.Input(shape=(2,))
    ])

  def call(self, inputs):
    user_id_input = inputs['user_id']
    country_input = inputs['country']
    age_input = inputs['age']
    gender_input = inputs['gender']
    device_input = inputs['device']
    # Print shape of inputs
    print("=================Shape of user inputs=====================")
    print(f"Shape of input user_id: {user_id_input.shape}")
    print(f"Shape of input country: {country_input.shape}")
    print(f"Shape of input age: {age_input.shape}")
    print(f"Shape of input gender: {gender_input.shape}")
    print(f"Shape of input device: {device_input.shape}")
    # Pass our inputs to models
    country_embedding = self.country(country_input)
    country_text_embedding = self.country_text(country_input)
    user_embedding = self.user(user_id_input)
    age_passed = self.age(age_input)
    device_passed = self.device(device_input)
    gender_passed = self.gender(gender_input)


    return tf.concat([
        country_embedding,
        country_text_embedding,
        user_embedding,
        age_passed,
        device_passed,
        gender_passed
    ], axis=1)

class QueryModel(tf.keras.Model):
  def __init__(self, layer_sizes):
    super().__init__()

    self.embedding_model = UserModel()

    self.dense_layers = tf.keras.Sequential()

    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation='relu'))

    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)

    return self.dense_layers(feature_embedding)

class MovieModel(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.video_id_embedding = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=max(unique_video_id) + 1, output_dim=768, input_length=1)
    ])
    self.video_duration= tf.keras.Sequential([
        tf.keras.Input(shape=())
    ])
    self.title = tf.keras.Sequential([
        tf.keras.Input(shape=(768,))
    ])


  def call(self, inputs):

    # Put inputs into variables
    video_id_input = inputs['video_id']
    title_input = inputs['title']
    video_duration_input = inputs['video_duration']

    # Print shape of inputs\
    print("========================Movie model input======================")
    print(f"Shape of video_id_input: {video_id_input.shape}")
    print(f"Shape of title_input: {title_input.shape}")
    print(f"Shape of video_duration_input: {video_duration_input.shape}")

    # Feed video_id_input into video_id_embedding model
    video_id_embedded = self.video_id_embedding(video_id_input)
    video_duration_passed = self.video_duration(video_duration_input)
    title_passed = self.title(title_input)

    # Print shape of passed data through layers
    print("==============Movie model embedding=====================")
    print(f"Shape of video_id_embedded: {video_id_embedded.shape}")
    print(f"Shape of video_duration_passed: {my_func(video_duration_passed).shape}")
    print(f"Shape of title_passed: {title_passed.shape}")

    concatenated_movie = tf.concat([
        video_id_embedded,
        my_func(video_duration_passed),
        title_passed
    ], axis=1)

    print("=================Concatenated movie model========================")
    print(f"Shape of concatenated movie model: {concatenated_movie.shape}")

    return concatenated_movie
class CandidateModel(tf.keras.Model):
  def __init__(self, layer_sizes):
    super().__init__()

    self.embedding_model = MovieModel()

    self.dense_layers = tf.keras.Sequential()

    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation='relu'))

    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

class MovielensModel(tfrs.models.Model):
  def __init__(self, layer_sizes: float, rating_weight: float, retrival_weight: float) -> None:
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)

    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=datasets.batch(8).map(self.candidate_model)
        )
    )

    self.rating_weight = rating_weight
    self.retrival_weight = retrival_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

    user_embedding = self.query_model({
        'user_id': features['user_id'],
        # 'country': features['country'],
        'age': features['age'],
        'gender': features['gender'],
        'device': features['device']
    })

    movie_embeddings = self.candidate_model({
        'video_id': features['video_id'],
        'title': features['title'],
        'video_duration': features['video_duration']
    })
    #Print some information
    print("===============MovieLensModel====================")
    print(f"Candidate Tower Shape: {movie_embedding.shape}")
    print(f"Query Tower shape: {user_embedding.shape}")

    return (
        user_embedding,
        movie_embeddings,
        self.rating_model(
            tf.concat([user_embedding, movie_embeddings], axis=1)
        )
    )

  def compute_loss(self, features, training=False):

    ratings = features.pop('watch_duration')

    user_embedding, movie_embeddings, rating_predictions = self(features)

    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions
    )
    retrieval_loss = self.retrieval_task(user_embedding, movie_embeddings, compute_merics=not training)

    return (
        self.rating_weight * rating_loss
        + self.retrival_weight * retrieval_loss
    )

In [77]:
model = MovielensModel(layer_sizes=[64, 32], rating_weight=1.0, retrival_weight=1.0)

model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1), run_eagerly=True)

history = model.fit(
    cached_train,
    validation_data=cached_eval,
    validation_freq=5,
    epochs=2
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


========================Movie model input======================
Shape of video_id_input: (None,)
Shape of title_input: (None, 768)
Shape of video_duration_input: (None,)
==============Movie model embedding=====================
Shape of video_id_embedded: (None, 768)
Shape of video_duration_passed: (None, 1)
Shape of title_passed: (None, 768)
=================Concatenated movie model========================
Shape of concatenated movie model: (None, 1537)
========================Movie model input======================
Shape of video_id_input: (None,)
Shape of title_input: (None, 768)
Shape of video_duration_input: (None,)
==============Movie model embedding=====================
Shape of video_id_embedded: (None, 768)
Shape of video_duration_passed: (None, 1)
Shape of title_passed: (None, 768)
=================Concatenated movie model========================
Shape of concatenated movie model: (None, 1537)
========================Movie model input======================
Shape of video_id_in

ValueError: Cannot convert '('c', 'o', 'u', 'n', 't', 'e', 'r')' to a shape. Found invalid entry 'c' of type '<class 'str'>'. 

In [ ]:
5# class UserModel(tf.keras.Model):
#   def __init__(self):
#     super().__init__()

#     self.user = tf.keras.Sequential([
#         tf.keras.layers.Embedding(input_dim=max(unique_user_id) + 1, output_dim=32, input_length=1)
#     ])
#     self.country = tf.keras.Sequential([
#         tf.keras.layers.StringLookup(
#             vocabulary=unique_country, mask_token=None
#         ),
#         tf.keras.layers.Embedding(len(unique_country) + 1, 32)
#     ])
#     self.age = tf.keras.Sequential([
#         tf.keras.Input(shape=(2,))
#     ])
#     self.device = tf.keras.Sequential([
#         tf.keras.Input(shape=(4,))
#     ])
#     self.gender = tf.keras.Sequential([
#         tf.keras.Input(shape=(2,))
#     ])

#   def call(self, inputs):
#     # put each input into an variable
#     user_id_input = inputs['user_id']
#     country_input = inputs['country']
#     age_input = inputs['age']
#     gender_input = inputs['gender']
#     device_input = inputs['device']
#     # Print some information
#     print(f"Shape of input user_id: {user_id_input.shape}")
#     print(f"Shape of input country: {country_input.shape}")
#     print(f"Shape of input age: {age_input.shape}")
#     print(f"Shape of input gender: {gender_input.shape}")
#     print(f"Shape of input device: {device_input.shape}")
#     # # Pass our inputs to models
#     country_embedding = self.country(country_input)
#     # print(f"Shape of embedding country: {country_embedding.shape}")
#     user_embedding = self.user(user_id_input)
#     # print(f"Shape of embedding user_id: {user_embedding.shape}")
#     age_passed = self.age(age_input)
#     # print(f"Shape of passed age: {age_passed.shape}")
#     gender_passed = self.gender(gender_input)
#     # print(f"Shape of passed gender: {gender_passed.shape}")
#     device_passed = self.device(device_input)
#     # print(f"Shape of passed device: {device_passed.shape}")
#     # Padding variable to have same shape and size
#     max_length_1 = 32

#     age_padded = tf.pad(age_passed, [[0, 0], [0, max_length_1 - tf.shape(age_passed)[1]]])
#     device_padded = tf.pad(device_passed, [[0, 0], [0, max_length_1 - tf.shape(device_passed)[1]]])
#     gender_padded = tf.pad(gender_passed, [[0, 0], [0, max_length_1 - tf.shape(gender_passed)[1]]])
#     country_padded = tf.pad(country_embedding, [[0, 0], [0, max_length_1 - tf.shape(country_embedding)[1]]])
#     user_id_padded = tf.pad(tf.squeeze(user_embedding), [[0, 0], [0, max_length_1 - tf.shape(tf.squeeze(user_embedding))[1]]])
#     # Print some information
#     # print(f"Shape of user_id afetr my_func: {my_func(user_id_padded).shape}")
#     # print(f"Shape of country afetr my_func: {my_func(country_padded).shape}")
#     # print(f"Shape of age afetr my_func: {my_func(age_padded).shape}")
#     # print(f"Shape of device afetr my_func: {my_func(device_padded).shape}")
#     # print(f"Shape of gender afetr my_func: {my_func(gender_padded).shape}")

#     return tf.concat([
#         my_func(user_id_padded),
#         my_func(country_padded),
#         my_func(age_padded),
#         my_func(gender_padded),
#         my_func(device_padded),
#     ], axis=1)

# class QueryModel(tf.keras.Model):
#   def __init__(self, layer_sizes):
#     super().__init__()

#     self.embedding_model = UserModel()

#     self.dense_layers = tf.keras.Sequential()

#     self.dense_layers.add(tf.keras.layers.Reshape())

#     for layer_size in layer_sizes[:-1]:
#       self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation='relu'))

#     for layer_size in layer_sizes[:-1]:
#       self.dense_layers.add(tf.keras.layers.Dense(layer_size))

#   def call(self, inputs):
#     feature_embedding = self.embedding_model(inputs)
#     query_output = self.dense_layers(feature_embedding)
#     # print(f"Shape of QueryTower: {query_output.shape}")

#     return query_output

# class MovieModel(tf.keras.Model):
#   def __init__(self):
#     super().__init__()

#     self.video_id = tf.keras.Sequential([
#         tf.keras.layers.Embedding(input_dim=max(unique_video_id) + 1, output_dim=768, input_length=1)
#     ])
#     self.video_duration= tf.keras.Sequential([
#         tf.keras.Input(shape=(1,))
#     ])
#     self.title = tf.keras.Sequential([
#         tf.keras.Input(shape=(768,))
#     ])

#   def call(self, inputs):

#     # Put inputs into variables
#     video_id_input = inputs['video_id']
#     title_input = inputs['title']
#     video_duration_input = inputs['video_duration']

#     # Convert video_duration input into useable format
#     video_duration_reformated = tf.reshape(tf.convert_to_tensor([tf.cast(video_duration_input, dtype=tf.float32)], dtype=tf.float32), [8, 1])

#     # Feed video_id_input into video_id_embedding model
#     video_id_embedding = self.video_id(video_id_input)
#     video_duration_passed = self.video_duration(video_duration_reformated)
#     title_passed = self.title(title_input)

#     # Padding variables to have same shape and size
#     max_length = 768  # Adjust this value as needed
#     video_id_padded = tf.pad(tf.squeeze(video_id_embedding), [[0, 0], [0, max_length - tf.shape(tf.squeeze(video_id_embedding))[1]]])
#     video_duration_padded = tf.pad(video_duration_passed, [[0, 0], [0, max_length - tf.shape(video_duration_passed)[1]]])
#     title_padded = tf.pad(title_passed, [[0, 0], [0, max_length - tf.shape(title_passed)[1]]])
#     # See some informations
#     # print(f"Shape of video_duration_padded after applying my_func: {my_func(video_duration_padded).shape}")
#     # print(f"Shape of video_id_padded after applying my_func: {my_func(video_id_padded).shape}")
#     # print(f"Shape of title_padded after applying my_func: {my_func(title_padded).shape}")
#     return tf.concat([
#         my_func(video_id_padded),
#         my_func(title_padded),
#         my_func(video_duration_padded)
#     ], axis=1)

# class CandidateModel(tf.keras.Model):
#   def __init__(self, layer_sizes):
#     super().__init__()

#     self.embedding_model = MovieModel()

#     self.dense_layers = tf.keras.Sequential()

#     for layer_size in layer_sizes[:-1]:
#       self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation='relu'))

#     for layer_size in layer_sizes[:-1]:
#       self.dense_layers.add(tf.keras.layers.Dense(layer_size))

#   def call(self, inputs):
#     feature_embedding = self.embedding_model(inputs)
#     candidate_output = self.dense_layers(feature_embedding)
#     # print(f"Shape of CandidateTower: {candidate_output.shape}")
#     return candidate_output

# class MovielensModel(tfrs.models.Model):
#   def __init__(self, layer_sizes: float, rating_weight: float, retrival_weight: float) -> None:
#     super().__init__()
#     self.query_model = QueryModel(layer_sizes)
#     self.candidate_model = CandidateModel(layer_sizes)

#     self.rating_model = tf.keras.Sequential([
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(16, activation='relu'),
#         tf.keras.layers.Dense(1)
#     ])

#     self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
#         loss=tf.keras.losses.MeanSquaredError(),
#         metrics=[tf.keras.metrics.RootMeanSquaredError()]
#     )

#     self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
#         metrics=tfrs.metrics.FactorizedTopK(
#             candidates=datasets.batch(8).map(self.candidate_model)
#         )
#     )

#     self.rating_weight = rating_weight
#     self.retrival_weight = retrival_weight

#   def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:

#     user_embedding = self.query_model({
#         'user_id': features['user_id'],
#         'country': features['country'],
#         'age': features['age'],
#         'gender': features['gender'],
#         'device': features['device']
#     })

#     movie_embeddings = self.candidate_model({
#         'video_id': features['video_id'],
#         'title': features['title'],
#         'video_duration': features['video_duration']
#     })


#     # Print shape of Towers
#     print(f"Shape of CandidateTower: {movie_embeddings.shape}")
#     print(f"Shape of QueryTower: {user_embedding.shape}")

#     return (
#         user_embedding,
#         movie_embeddings,
#         self.rating_model(
#             tf.concat([user_embedding, movie_embeddings], axis=1)
#         )
#     )

#   def compute_loss(self, features, training=False):

#     ratings = features.pop('watch_duration')

#     user_embedding, movie_embeddings, rating_predictions = self(features)

#     rating_loss = self.rating_task(
#         labels=ratings,
#         predictions=rating_predictions
#     )
#     retrieval_loss = self.retrieval_task(user_embedding, movie_embeddings, compute_metrics=not training)

#     return (
#         self.rating_weight * rating_loss
#         + self.retrival_weight * retrieval_loss
#     )